In [10]:
import feast # must be imported first, not sure why.
import quspin as qp
print(qp.__version__)

0.3.4


In [1]:
import feast # must be imported first, not sure why.
import quspin as qp
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d,boson_basis_1d # Hilbert space boson basis
import numpy as np # generic math functions
import time
from scipy.linalg import eigh

start=time.time()
##### define model parameters #####
L = 10 # system size
Nb=L//2
# sps=4
J = 1.0 # hopping strength
halfU = 1.0

basis=boson_basis_1d(L=L, Nb=Nb)
print(basis.Ns)

U_n=[[-halfU,i] for i in range(L)]
J_nn=[[J,i,i+1] for i in range(L-1)]
J_nn2=[[J,i,i+1] for i in range(L-1)]
U_int=[[halfU,i,i] for i in range(L)]
static=[["+-",J_nn],["-+",J_nn2],["nn",U_int],["n",U_n]]

###### construct Hamiltonian
H_b=hamiltonian(static,[],dtype=np.float64,basis=basis,check_herm=True)
H=H_b.static
spec,psil=eigh((H_b.static).todense())
print("scipy.linalg.eigh finished!")

import feast
### Working for real or complex Hermitian csr_matrix (Compressed Sparse Row matrix)
### M0:subspace size; Em:Energy_min; Ex:Energy_max; which:-1(find M0 lowest), 0(find in [Em,Ex]), 1(find M0 largest)
### default which=-1
### methods for HSolver class:
### .fpm: access fpm array
### .setfpm(cp=None,eps=None,ml=None,it=None): set fpm array
### .eigsh(debug=0): run FEAST 
### .eig(): access eigenvlaues
### .vec(): access eigenvectors
### .resid(): access residual 

# obj=feast.HSolver(H,M0=1) # find lowest 1 eigenvalue
obj=feast.HSolver(H,M0=40,Em=-9.0,Ex=-6.0,which=0) # find eigenvalues in spectrum [-9.0,-6.0]
# obj.setfpm(cp=16,eps=12,ml=2,it=1) # change default fpm parameter if needed
# M,info=obj.eigsh(se=2) # estimate of #eigenvalues, M0>=1.5M is suggested, change Em/Ex or rebuild obj if M0<1.5M.
egs,psi,M,info=obj.eigsh(debug=0)
if info == 0:
    print("FEAST sucess!",M,"eigenvlaues found.")
else:
    print("info =",info,"FEAST warning or failed!")
# info Classiﬁcation Description
# 202 Error Problem with size of the system N
# 201 Error Problem with size of subspace M0
# 200 Error Problem with Emin, Emax or Emid, r
# (100 + i) Error Problem with ithvalue of the input FEAST parameter (i.e fpm(i))
# 7 Warning The search for extreme eigenvalues has failed, search contour must be set by user
# 6 Warning FEAST converges but subspace is not bi-orthonormal
# 5 Warning Only stochastic estimation of #eigenvalues returned fpm(14)=2
# 4 Warning Only the subspace has been returned using fpm(14)=1
# 3 Warning Size of the subspace M0 is too small (M0<=M)
# 2 Warning No Convergence (#iteration loops>fpm(4))
# 1 Warning No Eigenvalue found in the search interval
# 0 Successful exit
# −1 Error Internal error conversion single/double
# −2 Error Internal error of the inner system solver in FEAST Driver interfaces
# −3 Error Internal error of the reduced eigenvalue solverPossible cause for Hermitian problem: matrix B may not be positive deﬁnite
# −(100 + i) Error Problem with the ithargument of the FEAST interface

2002
Hermiticity check passed!
Symmetry checks passed!
Particle conservation check passed!
scipy.linalg.eigh finished!
info = 5 FEAST warning or failed!


In [22]:
# possible input for .setfpm function
# cp: contour points, it=0,2,cp in (1 to 20, 24, 32, 40, 48, 56), it=1, any cp>0
# eps: convergence criteria = 10^(-eps) 
# ml: refinement loop, ml>0
# it: integration type, 0: Gauss; 1: Trapezoidal; 2: Zolotarev (better for continuous spectrum)
# lsp: mixed precision, 0:double; 1: single
# ifeast: automatic switch if ifeast=1, usually used when matrix is very large (>10^6)
# ifp: accuracy of ifeast, 10^(-ipf)
# ifl: loop of ifeast
# Em: set E_min
# Ex: set E_max
# ref Table 1 @ http://www.ecs.umass.edu/~polizzi/feast/doc.htm
obj.setfpm(cp=8,eps=12,ml=1,it=1,lsp=1,ifeast=0,ifp=1,ifl=40,Em=-9.0,Ex=-6.0)
obj.fpm

array([     0,     16,     12,      2,      0,      1,      5,     16,
         -111,      1,   -111,   -111,      0,      0,      2,      1,
         -111,     30,      0,     16,    100,     16,     40,      1,
           40,   2002,   -111,   -111,      1, 121142,     40,     10,
           16,   -111,   -111,      1,      0,      1,      0,      0,
            1,      1,      0,      0,      1,     40,      0,      0,
            0,   -111,   -111,   -111,   -111,   -111,   -111,   -111,
         -111,   -111,   -111,      0,   -111,   -111,   -111,      0],
      dtype=int32)

In [5]:
# parameters after FEAST finished
# [1:real/2:complex, M0, Emin, Emax, 0:interal/-1:lowest/1:largest, loop, espout]
obj.para_check()

[1, 40, -9.0, -6.0, 0, 1, 1.3737159558028603e-13]

In [6]:
# normalized
np.array([abs(psi[:,0].conj().dot(psi[:,0])) for i in range(M)])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [7]:
# those are eigenvectors
np.array([sum(abs(H.dot(psi[:,i])-egs[i]*psi[:,i])) for i in range(M)])

array([2.26967658e-12, 1.75396210e-12, 2.17157267e-12, 2.68372570e-12,
       1.91828452e-12, 1.92589653e-12, 2.20360351e-12, 3.04481921e-12,
       2.31712767e-12, 2.41792145e-12, 2.73904926e-12, 3.15769465e-12,
       4.04313459e-12, 3.75309043e-12, 5.29966009e-12, 1.23629397e-11])

In [8]:
# compare with scipy.linalg.eigh
np.array([sum(abs(H.dot(psil[:,i])-spec[i]*psil[:,i])) for i in range(M)])

array([3.35239851e-13, 3.06048956e-13, 3.29184162e-13, 3.15140016e-13,
       3.22300513e-13, 3.30774823e-13, 3.12938312e-13, 3.49347648e-13,
       3.64557532e-13, 3.72055770e-13, 3.72573707e-13, 3.30088384e-13,
       3.35051226e-13, 3.22121549e-13, 3.62281912e-13, 3.83500310e-13])

In [9]:
# compare eigenvalues from FEAST with scipy.linalg.eigh
egs-spec[:M]

array([ 3.55271368e-15, -7.99360578e-15, -7.99360578e-15, -7.99360578e-15,
       -6.21724894e-15,  1.77635684e-15,  3.55271368e-15, -2.66453526e-15,
       -1.77635684e-15, -1.77635684e-15, -1.77635684e-15, -6.21724894e-15,
        2.66453526e-15, -7.99360578e-15, -8.88178420e-16, -8.88178420e-16])

In [10]:
# residual of eigenvalues
obj.resid()

array([1.07149194e-14, 7.39283933e-15, 1.05858044e-14, 1.28942459e-14,
       8.74710644e-15, 9.71369208e-15, 1.02823865e-14, 1.44527594e-14,
       1.03277618e-14, 1.05108423e-14, 1.20062118e-14, 1.37786191e-14,
       1.76785112e-14, 1.63929650e-14, 2.64398235e-14, 6.37909412e-14])